# PyTorch Flowers Project

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import ToTensor
import scipy


In [23]:
from torchvision import transforms

myTransform = transforms.Compose([transforms.Resize(size=(500,500)), transforms.PILToTensor()])

#flowertensor = torchvision.io.read_image("C:/Users/megan/Desktop/flowers-102/jpg/image_00001.jpg")
train_flowers = torchvision.datasets.Flowers102(root="C:/Users/megan/Desktop", split="train", download=True, transform=myTransform)
test_flowers = torchvision.datasets.Flowers102(root="C:/Users/megan/Desktop", split="test", download=True, transform=myTransform)


https://pytorch.org/vision/stable/transforms.html

In [24]:
batch_size = 2

from torch.nn.utils.rnn import pad_sequence
#from torch.utils.data_utils import default_collate_fn_map

# Extend this function to handle batch of tensors
#def collate_tensor_fn(batch, *, collate_fn_map):
#    return torch.stack(batch, 0)
#def custom_collate(batch):
#    collate_map = {torch.Tensor: collate_tensor_fn}
#    return collate(batch, collate_fn_map=collate_map)
# Extend `default_collate` by in-place modifying `default_collate_fn_map`
#default_collate_fn_map.update({torch.Tensor: collate_tensor_fn})

# create data loader
train_dataloader = DataLoader(train_flowers,
                             # collate_fn=collate_fn,
                              batch_size=batch_size)

#train_dataloader = pad_sequence(DataLoader(train_flowers, batch_size=batch_size))
test_dataloader = DataLoader(test_flowers, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X []: {X.shape}")
    print(f"Shape of y: {y.dtype}")
    break


Shape of X []: torch.Size([2, 3, 500, 500])
Shape of y: torch.int64


In previous, tensor size is not equal, so it torch can't compare them, i think this is because the images aren't sized all the same and the number of dimensions of the 3rd parameter is inconsistent 

problem is in collate_fn 
https://pytorch.org/docs/stable/data.html#dataloader-collate-fn

In [30]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(500*500, 512), #second param must match first param of line 13
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=250000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [32]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [33]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [34]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 must have the same dtype